### ESMA 4016
### Regresion Logistica con las libreria H20  y scikit-learn
#### Edgar Acuna

In [186]:
import h2o
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.naive_bayes import H2ONaiveBayesEstimator
from h2o.estimators.glm   import H2OGeneralizedLinearEstimator

h2o.init()
h2o.no_progress()


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,10 hours 50 mins
H2O cluster timezone:,America/La_Paz
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.6
H2O cluster version age:,4 days
H2O cluster name:,H2O_from_python_edgar2017_7mwin8
H2O cluster total nodes:,1
H2O cluster free memory:,1.583 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


### I- Aplicando Naive Bayes a  Diabetes usando H20

In [187]:
diabetes = h2o.import_file("https://academic.uprm.edu/eacuna/diabetes.dat")
myx=['C1','C2','C3','C4','C5','C6','C7','C8']
diabetes['C9']=diabetes['C9'].asfactor()
myy="C9"
# Create test/train split
#train, test = vehicle.split_frame(ratios=[0.75], seed=1)
model=H2ONaiveBayesEstimator()
model.train(myx, myy, training_frame = diabetes)
y_pred=model.predict(diabetes)
print (y_pred['predict']==diabetes['C9']).sum()/float(len(diabetes))

0.72265625


### II -Aplicando Naive Bayes al dataset  Loan que tiene datos continuos y categoricos usando H20

In [188]:
loan = h2o.import_file("https://academic.uprm.edu/eacuna/datosarbol.csv")
loan

Sexo,Familia,CasPropia,AnosEmpleo,Sueldo,StatustMarital,Prestamo
Hombre,3,No,17,2500,Soltero,No
Mujer,5,Si,10,3000,Casado,Si
Mujer,4,No,15,2000,Viudo,No
Hombre,3,Si,16,2800,Soltero,Si
Hombre,6,Si,11,4000,Viudo,Si
Mujer,4,Si,26,3200,Soltero,Si
Mujer,2,Si,14,1800,Soltero,No
Hombre,5,Si,10,3750,Casado,Si
Hombre,6,No,18,2970,Divorciado,No
Hombre,4,Si,12,3350,Divorciado,No


In [189]:
myx=['Sexo','Familia','CasPropia', 'AnosEmpleo','Sueldo','StatustMarital']
loan['Prestamo']=loan['Prestamo'].asfactor()
myy="Prestamo"
# Create test/train split
#train, test = vehicle.split_frame(ratios=[0.75], seed=1)
model=H2ONaiveBayesEstimator()
model.train(myx, myy, training_frame = loan)
y_pred=model.predict(loan)
print (y_pred['predict']==loan['Prestamo']).sum()/float(len(loan))
model.model_performance(loan)

0.88

ModelMetricsBinomial: naivebayes
** Reported on test data. **

MSE: 0.0854946728279
RMSE: 0.29239472093
LogLoss: 0.277825390866
Mean Per-Class Error: 0.1
AUC: 0.96
Gini: 0.92
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.458241448484: 


,No,Si,Error,Rate
No,7.0,3.0,0.3,(3.0/10.0)
Si,0.0,15.0,0.0,(0.0/15.0)
Total,7.0,18.0,0.12,(3.0/25.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4582414,0.9090909,17.0
max f2,0.4582414,0.9615385,17.0
max f0point5,0.6756497,0.9523810,11.0
max accuracy,0.6756497,0.88,11.0
max precision,0.9996507,1.0,0.0
max recall,0.4582414,1.0,17.0
max specificity,0.9996507,1.0,0.0
max absolute_mcc,0.6756497,0.7844645,11.0
max min_per_class_accuracy,0.5195887,0.8666667,13.0
max mean_per_class_accuracy,0.6756497,0.9,11.0


Gains/Lift Table: Avg response rate: 60.00 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.04,0.9995576,1.6666667,1.6666667,1.0,1.0,0.0666667,0.0666667,66.6666667,66.6666667
,2,0.04,0.9994645,0.0,1.6666667,0.0,1.0,0.0,0.0666667,-100.0,66.6666667
,3,0.04,0.9993714,0.0,1.6666667,0.0,1.0,0.0,0.0666667,-100.0,66.6666667
,4,0.04,0.9992784,0.0,1.6666667,0.0,1.0,0.0,0.0666667,-100.0,66.6666667
,5,0.08,0.9992516,1.6666667,1.6666667,1.0,1.0,0.0666667,0.1333333,66.6666667,66.6666667
,6,0.12,0.9990432,1.6666667,1.6666667,1.0,1.0,0.0666667,0.2,66.6666667,66.6666667
,7,0.16,0.9980499,1.6666667,1.6666667,1.0,1.0,0.0666667,0.2666667,66.6666667,66.6666667
,8,0.2,0.9644366,1.6666667,1.6666667,1.0,1.0,0.0666667,0.3333333,66.6666667,66.6666667
,9,0.32,0.8380636,1.6666667,1.6666667,1.0,1.0,0.2,0.5333333,66.6666667,66.6666667
,10,0.4,0.7071564,1.6666667,1.6666667,1.0,1.0,0.1333333,0.6666667,66.6666667,66.6666667


### III. Aplicando Naive Bayes  al dataset  Vehicle usando H20

In [190]:
#Aplicando Naive Baye  a vehicle
vehicle = h2o.import_file("https://academic.uprm.edu/eacuna/vehicle.csv")
myx=['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18']
vehicle['class']=vehicle['class'].asfactor()
myy="class"
# Create test/train split
#train, test = vehicle.split_frame(ratios=[0.75], seed=1)
model=H2ONaiveBayesEstimator()
model.train(myx, myy, training_frame = vehicle)
y_pred=model.predict(vehicle)
print (y_pred['predict']==vehicle['class']).sum()/float(len(vehicle))
model.model_performance(vehicle)

0.472813238771

ModelMetricsMultinomial: naivebayes
** Reported on test data. **

MSE: 0.435659850789
RMSE: 0.660045339949
LogLoss: 2.39864325324
Mean Per-Class Error: 0.51830069164
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



1,2,3,4,Error,Rate
39.0,35.0,11.0,133.0,0.8211009,179 / 218
1.0,93.0,55.0,63.0,0.5613208,119 / 212
2.0,58.0,90.0,67.0,0.5852535,127 / 217
10.0,3.0,8.0,178.0,0.1055276,21 / 199
52.0,189.0,164.0,441.0,0.5271868,446 / 846


Top-4 Hit Ratios: 


k,hit_ratio
1,0.4728133
2,0.8120568
3,0.9503546
4,1.0


### IV. Aplicando Regresion Logistica al dataset loan usando h20

In [191]:
loan['Sexo'] = loan['Sexo'].asfactor()
loan["CasPropia"]=loan['CasPropia'].asfactor()
loan['StatustMarital']=loan['StatustMarital'].asfactor()
loan['Prestamo']=loan['Prestamo'].asfactor()

predictors = ["Sexo","Familia", "CasPropia","AnosEmpleo", "Sueldo","StatustMarital"]
response_col = "Prestamo"

In [192]:
loan.head()

Sexo,Familia,CasPropia,AnosEmpleo,Sueldo,StatustMarital,Prestamo
Hombre,3,No,17,2500,Soltero,No
Mujer,5,Si,10,3000,Casado,Si
Mujer,4,No,15,2000,Viudo,No
Hombre,3,Si,16,2800,Soltero,Si
Hombre,6,Si,11,4000,Viudo,Si
Mujer,4,Si,26,3200,Soltero,Si
Mujer,2,Si,14,1800,Soltero,No
Hombre,5,Si,10,3750,Casado,Si
Hombre,6,No,18,2970,Divorciado,No
Hombre,4,Si,12,3350,Divorciado,No


In [193]:
glm_model = H2OGeneralizedLinearEstimator(family= "binomial", lambda_ = 0, compute_p_values = True)
glm_model.train(predictors, response_col, training_frame= loan)

# Coefficients that can be applied to the non-standardized data.
print(glm_model.coef())

{u'Familia': -7.283161660798802, u'CasPropia.Si': -1.4732452885828509, u'AnosEmpleo': 2.9574452082674885, u'Sueldo': 0.09868220640412526, u'Intercept': -269.16800091477114, u'Sexo.Mujer': -4.160890209638011, u'StatustMarital.Soltero': -18.717097643950506, u'StatustMarital.Divorciado': -102.24457434194633, u'StatustMarital.Viudo': -10.697821309701698}


In [194]:
glm_model

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1524120852419_150


ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 1.98476633847e-12
RMSE: 1.40881735455e-06
LogLoss: 5.98743562425e-07
Null degrees of freedom: 24
Residual degrees of freedom: 16
Null deviance: 33.6505833505
Residual deviance: 2.99371781214e-05
AIC: 18.0000299372
AUC: 1.0
Gini: 1.0
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.999995968921: 


,No,Si,Error,Rate
No,10.0,0.0,0.0,(0.0/10.0)
Si,0.0,15.0,0.0,(0.0/15.0)
Total,10.0,15.0,0.0,(0.0/25.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.9999960,1.0,7.0
max f2,0.9999960,1.0,7.0
max f0point5,0.9999960,1.0,7.0
max accuracy,0.9999960,1.0,7.0
max precision,1.0,1.0,0.0
max recall,0.9999960,1.0,7.0
max specificity,1.0,1.0,0.0
max absolute_mcc,0.9999960,1.0,7.0
max min_per_class_accuracy,0.9999960,1.0,7.0
max mean_per_class_accuracy,0.9999960,1.0,7.0


Gains/Lift Table: Avg response rate: 60.00 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.32,1.0,1.6666667,1.6666667,1.0,1.0,0.5333333,0.5333333,66.6666667,66.6666667
,2,0.4,1.0,1.6666667,1.6666667,1.0,1.0,0.1333333,0.6666667,66.6666667,66.6666667
,3,0.52,0.9999994,1.6666667,1.6666667,1.0,1.0,0.2,0.8666667,66.6666667,66.6666667
,4,0.6,0.5999992,1.6666667,1.6666667,1.0,1.0,0.1333333,1.0,66.6666667,66.6666667
,5,0.68,0.0000008,0.0,1.4705882,0.0,0.8823529,0.0,1.0,-100.0,47.0588235
,6,0.8,0.0000000,0.0,1.25,0.0,0.75,0.0,1.0,-100.0,25.0
,7,0.88,0.0000000,0.0,1.1363636,0.0,0.6818182,0.0,1.0,-100.0,13.6363636
,8,1.0,0.0000000,0.0,1.0,0.0,0.6,0.0,1.0,-100.0,0.0



Scoring History: 


,timestamp,duration,iterations,negative_log_likelihood,objective
,2018-04-19 13:44:22,0.000 sec,0,16.8252917,0.6730117
,2018-04-19 13:44:22,0.000 sec,1,5.7156605,0.2286264
,2018-04-19 13:44:22,0.001 sec,2,3.2640929,0.1305637
,2018-04-19 13:44:22,0.001 sec,3,1.9230352,0.0769214
,2018-04-19 13:44:22,0.001 sec,4,1.0374345,0.0414974
---,---,---,---,---,---
,2018-04-19 13:44:22,0.005 sec,46,0.0000168,0.0000007
,2018-04-19 13:44:22,0.005 sec,47,0.0000163,0.0000007
,2018-04-19 13:44:22,0.006 sec,48,0.0000158,0.0000006
,2018-04-19 13:44:22,0.006 sec,49,0.0000154,0.0000006



See the whole table with table.as_data_frame()


In [195]:
y_pred=glm_model.predict(loan)
print (y_pred['predict']==loan['Prestamo']).sum()/float(len(loan))

1.0


### IV- Regresion Logistica para Diabetes usando H20

In [196]:
diabetes = h2o.import_file("https://academic.uprm.edu/eacuna/diabetes.dat")
myx=['C1','C2','C3','C4','C5','C6','C7','C8']
diabetes['C9']=diabetes['C9'].asfactor()
myy='C9'
glm_model = H2OGeneralizedLinearEstimator(family= "binomial", lambda_ = 0, compute_p_values = True)
glm_model.train(myx, myy, training_frame= diabetes)
glm_model

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1524120852419_153


ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.152725755701
RMSE: 0.390801427455
LogLoss: 0.470993084488
Null degrees of freedom: 767
Residual degrees of freedom: 759
Null deviance: 993.483910139
Residual deviance: 723.445377774
AIC: 741.445377774
AUC: 0.839358208955
Gini: 0.67871641791
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.354383535808: 


,1,2,Error,Rate
1,392.0,108.0,0.216,(108.0/500.0)
2,70.0,198.0,0.2612,(70.0/268.0)
Total,462.0,306.0,0.2318,(178.0/768.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3543835,0.6898955,199.0
max f2,0.1507638,0.8010013,297.0
max f0point5,0.6475871,0.7102273,102.0
max accuracy,0.4867680,0.7851562,148.0
max precision,0.9925888,1.0,0.0
max recall,0.0136847,1.0,392.0
max specificity,0.9925888,1.0,0.0
max absolute_mcc,0.4867680,0.5109421,148.0
max min_per_class_accuracy,0.3310636,0.7537313,207.0
max mean_per_class_accuracy,0.3543835,0.7614030,199.0


Gains/Lift Table: Avg response rate: 34.90 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0104167,0.9549975,2.1492537,2.1492537,0.75,0.75,0.0223881,0.0223881,114.9253731,114.9253731
,2,0.0208333,0.9295637,2.5074627,2.3283582,0.875,0.8125,0.0261194,0.0485075,150.7462687,132.8358209
,3,0.03125,0.9049045,2.5074627,2.3880597,0.875,0.8333333,0.0261194,0.0746269,150.7462687,138.8059701
,4,0.0403646,0.8895109,2.4562900,2.4034665,0.8571429,0.8387097,0.0223881,0.0970149,145.6289979,140.3466538
,5,0.0507812,0.8736843,2.5074627,2.4247991,0.875,0.8461538,0.0261194,0.1231343,150.7462687,142.4799082
,6,0.1002604,0.7793791,2.4886096,2.4562900,0.8684211,0.8571429,0.1231343,0.2462687,148.8609584,145.6289979
,7,0.1510417,0.7093225,2.2778416,2.3962944,0.7948718,0.8362069,0.1156716,0.3619403,127.7841561,139.6294390
,8,0.2005208,0.6444353,2.1115475,2.3260322,0.7368421,0.8116883,0.1044776,0.4664179,111.1547526,132.6032177
,9,0.3007812,0.4581469,1.3397945,1.9972863,0.4675325,0.6969697,0.1343284,0.6007463,33.9794534,99.7286296
,10,0.3997396,0.3529273,1.3951296,1.8482182,0.4868421,0.6449511,0.1380597,0.7388060,39.5129615,84.8218192



Scoring History: 


,timestamp,duration,iterations,negative_log_likelihood,objective
,2018-04-19 13:44:22,0.000 sec,0,496.7419551,0.6467994
,2018-04-19 13:44:22,0.001 sec,1,370.8909597,0.4829309
,2018-04-19 13:44:22,0.002 sec,2,362.0294357,0.4713925
,2018-04-19 13:44:22,0.002 sec,3,361.7232470,0.4709938
,2018-04-19 13:44:22,0.003 sec,4,361.7226889,0.4709931
,2018-04-19 13:44:22,0.004 sec,5,361.7226889,0.4709931


In [197]:
glm_model._model_json['output']['coefficients_table']

Coefficients: glm coefficients



names,coefficients,std_error,z_value,p_value,standardized_coefficients
Intercept,-8.4046964,0.7166361,-11.7279840,0.0,-0.8711017
C1,0.1231823,0.0320776,3.8401399,0.0001230,0.4150724
C2,0.0351637,0.0037087,9.4813920,0.0,1.1242760
C3,-0.0132955,0.0052336,-2.5404157,0.0110721,-0.2573460
C4,0.0006190,0.0068994,0.0897131,0.9285152,0.0098739
C5,-0.0011917,0.0009012,-1.3223092,0.1860652,-0.1373362
C6,0.0897010,0.0150876,5.9453328,0.0000000,0.7072168
C7,0.9451797,0.2991475,3.1595776,0.0015800,0.3131651
C8,0.0148690,0.0093348,1.5928583,0.1111920,0.1748629


### VII Regresion Logistica para Diabetes usando scikit learn

In [198]:
url= "http://academic.uprm.edu/eacuna/diabetes.dat"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_table(url, names=names,header=None)
y=data['class']
X=data.iloc[:,0:8]
#Haciendo la regresion logistica ya calculando su precision
model = LogisticRegression()
model = model.fit(X, y)
print model.coef_

[[  1.17057358e-01   2.83881555e-02  -1.68873034e-02   7.55876596e-04
   -6.42924257e-04   5.97710296e-02   6.77534932e-01   7.24147639e-03]]


In [199]:
# Tasa de precision
model.score(X, y)
predictions = model.predict(X)
print(classification_report(y, predictions))

             precision    recall  f1-score   support

          1       0.79      0.90      0.84       500
          2       0.74      0.55      0.63       268

avg / total       0.77      0.77      0.77       768



### VIII Regresion Logistica para Loan usando scikit-learn 

In [200]:
data = pd.read_csv("https://academic.uprm.edu/eacuna/datosarbol.csv")
y=data['Prestamo']
lb_make = LabelEncoder()
data["Sexo"] = lb_make.fit_transform(data["Sexo"])
data['CasPropia']=lb_make.fit_transform(data["CasPropia"])
data['StatustMarital']=lb_make.fit_transform(data["StatustMarital"])
data['Prestamo']=lb_make.fit_transform(data["Prestamo"])
y2=data['Prestamo']
X=data.iloc[:,0:6]
#Haciendo la regresion logistica ya calculando su precision
model = LogisticRegression()
model = model.fit(X, y2)

In [201]:
# Tasa de precision
model.score(X, y2)
predictions = model.predict(X)
print(classification_report(y2, predictions))

             precision    recall  f1-score   support

          0       1.00      0.30      0.46        10
          1       0.68      1.00      0.81        15

avg / total       0.81      0.72      0.67        25

